In [1]:
from yeastdnnexplorer.probability_models.generate_data import (generate_gene_population, 
                                                               generate_binding_effects,
                                                               generate_pvalues,
                                                               generate_perturbation_effects)

import torch

torch.manual_seed(42)  # For CPU
torch.cuda.manual_seed_all(42)  # For all CUDA devices


## Generating a Set of Data

### Step 1:

The first step is to generate a gene population, or set of gene populations.
A gene population is simply a class that stores a 1D tensor called `labels`.
`labels` is a boolean vector where 1 means the gene is part of the signal group
(a gene which is both bound and responsive to the TF) while 0 means the gene is
part of the background or noise group. The length of `labels` is the number of
genes in the population, and the index should be considered the unique gene
identifier. In other words, the indicies should never change.

In [2]:
n_genes = 1000
signal = [0.1, 0.15, 0.2, 0.25, 0.3]
n_sample = [1, 1, 2, 2, 4]

# this will be a list of length 10 with a GenePopulation object in each element
gene_populations_list = []
for signal_proportion, n_draws in zip(signal, n_sample):
    for _ in range(n_draws):
        gene_populations_list.append(generate_gene_population(n_genes, signal_proportion))


### Step 2:

The second step is to generate binding data from the gene population(s).

In [3]:
# Generate binding data for each gene population
binding_effect_list = [generate_binding_effects(gene_population)
                     for gene_population in gene_populations_list]


# Calculate p-values for binding data
binding_pvalue_list = [generate_pvalues(binding_data) for binding_data in binding_effect_list]

binding_data_combined = [torch.stack((gene_population.labels, binding_effect, binding_pval), dim=1)
                         for gene_population, binding_effect, binding_pval
                         in zip (gene_populations_list, binding_effect_list, binding_pvalue_list)]

# Stack along a new dimension (dim=1) to create a tensor of shape [num_genes, num_TFs, 3]
binding_data_tensor = torch.stack(binding_data_combined, dim=1)

# Verify the shape
print("Shape of the binding data tensor:", binding_data_tensor.shape)


Shape of the binding data tensor: torch.Size([1000, 10, 3])


### Step 3: Generate perturbation data.

Note that you can optionally use the binding enrichment data across one or more
TFs to probabilistically increase the chance of a larger perturbation effect.

In [4]:
# note that you could use different settings for the perturbation effects
# for each TF, or possibly use the n_samples vector from step 1. You also have
# the option of using all or some of the TFs to conditionally affect the
# binding score. See `generate_perturbation_effects()` in the help or the
# documentation for more details.
perturbation_effects_list = [generate_perturbation_effects(binding_data_tensor)
                             for _ in range(sum(n_sample))]

perturbation_pvalue_list = [generate_pvalues(perturbation_effects)
                            for perturbation_effects in perturbation_effects_list]



### Step 4: Assemble

The final step is to assemble the data into a single tensor. Here is one way.
The order of the matrix in the last dimension is:

1. signal/noise label
1. binding effect
1. binding pvalue
1. perturbation effect
1. perturbation pvalue

In [5]:
# Convert lists to tensors if they are not already
perturbation_effects_tensor = torch.stack(perturbation_effects_list, dim=1)
perturbation_pvalues_tensor = torch.stack(perturbation_pvalue_list, dim=1)

# Ensure perturbation data is reshaped to match [n_genes, n_tfs]
# This step might need adjustment based on the actual shapes of your tensors.
perturbation_effects_tensor = perturbation_effects_tensor.unsqueeze(-1)  # Adds an extra dimension for concatenation
perturbation_pvalues_tensor = perturbation_pvalues_tensor.unsqueeze(-1)  # Adds an extra dimension for concatenation

# Concatenate along the last dimension to form a [n_genes, n_tfs, 5] tensor
final_data_tensor = torch.cat((binding_data_tensor, perturbation_effects_tensor, perturbation_pvalues_tensor), dim=2)

# Verify the shape
print("Shape of the final data tensor:", final_data_tensor.shape)

Shape of the final data tensor: torch.Size([1000, 10, 5])


As an aside, I choose to structure the data this way by looking at the
result of strides, which describes how the data is stored in memory:

In [6]:
tensor_continuous = torch.empty(100, 1000, 3)
strides_continuous = tensor_continuous.stride()
print(strides_continuous)


tensor_continuous = torch.empty(1000, 100, 3)
strides_continuous = tensor_continuous.stride()
print(strides_continuous)

(3000, 3, 1)
(300, 3, 1)
